Notebook used to inspect results of Granger Causality analysis

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import math
import sys
from pathlib import Path
import re
from itertools import combinations
# Get the project root: notebooks/AI_narrative_index
root_dir = Path.cwd().parent

# Add needed folders to the Python modules search path
sys.path.append(str(root_dir / "src" / "scripts"))
sys.path.append(str(root_dir / "src" / "visualizations"))
sys.path.append(str(root_dir / "src" / "modelling"))

# import custom functions
#rom plot_granger_causality import plot_aini_lags_by_year, plot_aini_lags_for_year
from plot_functions import plot_n_articles_with_extrema_events, plot_stock_growth
from construct_tables import export_regression_table 
from compute_rejections import compute_rejection_rates_all, add_trading_days_columns, export_rejection_rates_to_pptx_all_tables_only
from concat_stationarity import concatenate_stationarity_results_to_html


In [2]:
concatenate_stationarity_results_to_html(
    output_basename="stationarity_all_vars",
    search_glob="stationarity_tests_*.csv",
    round_stats=3,
    drop_notes=True,
    truncate={"Variable": 28},   # (Label is dropped from final anyway)
    title="Stationarity tests (ADF, PP, KPSS) across all measures",
    verbose=True,                # prints before/after counts + fallback warning if needed
)


[HTML] Collected rows: 1704 -> after filter: 0
[HTML] WARNING: Filter removed all rows. Falling back to UNFILTERED output.
Saved master HTML -> C:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\reports\tables\stationarity_all_vars.html


WindowsPath('C:/Users/PC/Desktop/Masterarbeit/AI_narrative_index/reports/tables/stationarity_all_vars.html')

No control

In [ ]:
# define path to variables
var_path = root_dir / "data" / "processed" / "variables"
 
# load data (S&P 500 control)
gc_c = pd.read_csv(var_path / "granger_causality_binary.csv")
gc_w0 = pd.read_csv(var_path / "granger_causality_w0.csv")
gc_w1 = pd.read_csv(var_path / "granger_causality_w1.csv")
gc_w2 = pd.read_csv(var_path / "granger_causality_w2.csv")


# create column to indicate version
gc_c["Model"] = "custom"
gc_w0["Model"] = "w0"
gc_w1["Model"] = "w1"
gc_w2["Model"] = "w2"

# merge them together
gc_all_results = pd.concat([gc_c, gc_w0, gc_w1, gc_w2], ignore_index=True)
gc_all_results["joint rej. (α=0.1)"] = gc_all_results["BH_reject_F"] & gc_all_results["BH_reject_F_HC3"]

rename_map = {
    "p_x": "Lags",
    "BH_corr_F_pval": "BH empirical p",
    "BH_corr_F_pval_HC3": "BH analytical p",
    "Year": "Period"
}

# Add lag-based renames (A2R and R2A)
for i in range(1, 4):
    rename_map[f"A2R_beta_ret_{i}"] = f"β{i}"
    rename_map[f"A2R_beta_x_{i}"] = f"γ{i}"
    rename_map[f"R2A_beta_ret_{i}"] = f"β{i}"
    rename_map[f"R2A_beta_x_{i}"] = f"γ{i}"

# Apply renaming
gc_all_results = gc_all_results.rename(columns=rename_map)
gc_all_results


gc_all_results_for_report = gc_all_results.copy()
gc_all_results_for_report.columns

In [ ]:
gc_all_results_for_report_a2r = gc_all_results_for_report[gc_all_results_for_report["Direction"] == "AINI_to_RET"]
gc_all_results_for_report_a2r = gc_all_results_for_report_a2r.dropna(axis=1, how='all')
gc_all_results_for_report_a2r

In [ ]:
# HTML output
export_regression_table(
    df=gc_all_results_for_report_a2r,
    title="Granger-Causality all Results (AINI → Returns)",
    output_filename="gc_sp500_aini_to_ret",
    output_format="html"
)

In [ ]:
# calculate rejection rate 

# Make sure Year and Ticker are strings
gc_all_results_for_report_a2r["Period"] = gc_all_results_for_report_a2r["Period"].astype(str)
gc_all_results_for_report_a2r["Ticker"] = gc_all_results_for_report_a2r["Ticker"].astype(str)

# Total number of models tested
total = gc_all_results_for_report_a2r["joint rej. (α=0.1)"].count()

# Number of rejections (both bootstrap + HC3 significant)
n_reject = gc_all_results_for_report_a2r["joint rej. (α=0.1)"].sum()

# Rejection rate
rejection_rate = n_reject / total * 100

print(f"Total models: {total}")
print(f"Both-method rejections: {n_reject}")
print(f"Rejection rate: {rejection_rate:.2f}%")

Controlled for sox

In [ ]:
# define path to variables
var_path = root_dir / "data" / "processed" / "variables"
 
# load data (S&P 500 control)
gc_c_sox = pd.read_csv(var_path / "granger_causality_log_growth_sox_binary.csv")
gc_w0_sox = pd.read_csv(var_path / "granger_causality_log_growth_sox_w0.csv")
gc_w1_sox = pd.read_csv(var_path / "granger_causality_log_growth_sox_w1.csv")
gc_w2_sox = pd.read_csv(var_path / "granger_causality_log_growth_sox_w2.csv")


# create column to indicate version
gc_c_sox["Model"] = "custom"
gc_w0_sox["Model"] = "w0"
gc_w1_sox["Model"] = "w1"
gc_w2_sox["Model"] = "w2"

# merge them together
gc_all_results_sox = pd.concat([gc_c_sox, gc_w0_sox, gc_w1_sox, gc_w2_sox], ignore_index=True)
gc_all_results_sox["joint rej. (α=0.1)"] = gc_all_results_sox["BH_reject_F"] & gc_all_results_sox["BH_reject_F_HC3"]

rename_map = {
    "p_x": "Lags",
    "BH_corr_F_pval": "BH empirical p",
    "BH_corr_F_pval_HC3": "BH analytical p",
    "Year": "Period"
}

# Add lag-based renames (A2R and R2A)
for i in range(1, 4):
    rename_map[f"A2R_beta_ret_{i}"] = f"β{i}"
    rename_map[f"A2R_beta_x_{i}"] = f"γ{i}"
    rename_map[f"R2A_beta_ret_{i}"] = f"β{i}"
    rename_map[f"R2A_beta_x_{i}"] = f"γ{i}"

# Apply renaming
gc_all_results_sox = gc_all_results_sox.rename(columns=rename_map)
gc_all_results_sox

gc_all_results_for_report_sox = gc_all_results_sox.copy()
gc_all_results_for_report_sox

In [ ]:
gc_all_results_for_report_a2r_sox = gc_all_results_for_report_sox[gc_all_results_for_report_sox["Direction"] == "AINI_to_RET"]
gc_all_results_for_report_a2r_sox = gc_all_results_for_report_a2r_sox.dropna(axis=1, how='all')
gc_all_results_for_report_a2r_sox

In [ ]:
# HTML output
export_regression_table(
    df=gc_all_results_for_report_a2r_sox,
    title="Granger-Causality all Results (AINI → Returns, controlled for PHLX Index)",
    output_filename="gc_sp500_aini_to_ret_PHLX",
    output_format="html"
)
gc_all_results_for_report_a2r_sox

In [ ]:
# calculate rejection rate 

# Make sure Year and Ticker are strings
gc_all_results_for_report_a2r_sox["Period"] = gc_all_results_for_report_a2r_sox["Period"].astype(str)
gc_all_results_for_report_a2r_sox["Ticker"] = gc_all_results_for_report_a2r_sox["Ticker"].astype(str)

# Total number of models tested
total = gc_all_results_for_report_a2r_sox["joint rej. (α=0.1)"].count()

# Number of rejections (both bootstrap + HC3 significant)
n_reject = gc_all_results_for_report_a2r_sox["joint rej. (α=0.1)"].sum()

# Rejection rate
rejection_rate = n_reject / total * 100

print(f"Total models: {total}")
print(f"Both-method rejections: {n_reject}")
print(f"Rejection rate: {rejection_rate:.2f}%")

Controlled for S&P 500

In [ ]:
# define path to variables
var_path = root_dir / "data" / "processed" / "variables"
 
# load data (S&P 500 control)
gc_c = pd.read_csv(var_path / "granger_causality_log_growth_sp500_binary.csv")
gc_w0 = pd.read_csv(var_path / "granger_causality_log_growth_sp500_w0.csv")
gc_w1 = pd.read_csv(var_path / "granger_causality_log_growth_sp500_w1.csv")
gc_w2 = pd.read_csv(var_path / "granger_causality_log_growth_sp500_w2.csv")


# create column to indicate version
gc_c["Model"] = "custom"
gc_w0["Model"] = "w0"
gc_w1["Model"] = "w1"
gc_w2["Model"] = "w2"

# merge them together
gc_all_results = pd.concat([gc_c, gc_w0, gc_w1, gc_w2], ignore_index=True)
gc_all_results["joint rej. (α=0.1)"] = gc_all_results["BH_reject_F"] & gc_all_results["BH_reject_F_HC3"]

rename_map = {
    "p_x": "Lags",
    "BH_corr_F_pval": "BH empirical p",
    "BH_corr_F_pval_HC3": "BH analytical p",
    "Year": "Period"
}

# Add lag-based renames (A2R and R2A)
for i in range(1, 4):
    rename_map[f"A2R_beta_ret_{i}"] = f"β{i}"
    rename_map[f"A2R_beta_x_{i}"] = f"γ{i}"
    rename_map[f"R2A_beta_ret_{i}"] = f"β{i}"
    rename_map[f"R2A_beta_x_{i}"] = f"γ{i}"

# Apply renaming
gc_all_results_sp500 = gc_all_results.rename(columns=rename_map)
gc_all_results_sp500

gc_all_results_sp500_for_report = gc_all_results_sp500.copy()
gc_all_results_sp500_for_report

In [ ]:
gc_all_results_sp500_for_report_a2r = gc_all_results_sp500_for_report[gc_all_results_sp500_for_report["Direction"] == "AINI_to_RET"]
gc_all_results_sp500_for_report_a2r = gc_all_results_sp500_for_report_a2r.dropna(axis=1, how='all')
gc_all_results_sp500_for_report_a2r

In [ ]:
# calculate rejection rate 

# Make sure Year and Ticker are strings
gc_all_results_sp500_for_report_a2r["Period"] = gc_all_results_sp500_for_report_a2r["Period"].astype(str)
gc_all_results_sp500_for_report_a2r["Ticker"] = gc_all_results_sp500_for_report_a2r["Ticker"].astype(str)

# Total number of models tested
total = gc_all_results_sp500_for_report_a2r["joint rej. (α=0.1)"].count()

# Number of rejections (both bootstrap + HC3 significant)
n_reject = gc_all_results_sp500_for_report_a2r["joint rej. (α=0.1)"].sum()

# Rejection rate
rejection_rate = n_reject / total * 100

print(f"Total models: {total}")
print(f"Both-method rejections: {n_reject}")
print(f"Rejection rate: {rejection_rate:.2f}%")


In [ ]:
# HTML output
export_regression_table(
    df=gc_all_results_sp500_for_report_a2r,
    title="Granger-Causality all Results (AINI → Returns, controlled for S&P500)",
    output_filename="gc_sp500_aini_to_ret",
    output_format="html"
)

In [ ]:
# save merged results
gc_all_results_sp500.to_csv(var_path / "granger_causality_all_SP500.csv", index=False)

# define table path
table_path = root_dir / "reports" / "tables"

# Export as HTML for online appendix
gc_all_results.to_html(table_path / "granger_causality_SP500.html", index=False)
gc_all_results

In [ ]:
# subset by direction
sp500_aini_to_ret = gc_all_results_sp500[gc_all_results_sp500["Direction"] == "AINI_to_RET"]
sp500_ret_to_aini = gc_all_results_sp500[gc_all_results_sp500["Direction"] == "RET_to_AINI"]

# cols to keep
keep_a2r = [
    "Model", "AINI_variant", "Ticker", "Period", "Lags", "β₀",
    "β1", "β2", "β3",
    "γ1", "γ2", "γ3",
    "BH empirical p", "BH analytical p", "joint rej. (α=0.1)"
]

keep_r2a = [
    "Model", "AINI_variant", "Ticker", "Period", "Lags", "β₀",
    "β1", "β2", "β3",
    "γ1", "γ2", "γ3",
    "BH empirical p", "BH analytical p", "joint rej. (α=0.1)"
]

# subset
sp500_aini_to_ret_sub = sp500_aini_to_ret[keep_a2r]
sp500_ret_to_aini_sub = sp500_ret_to_aini[keep_r2a]



In [ ]:
sp500_aini_to_ret_sub

In [ ]:
# calculate rejection rate 

# Make sure Year and Ticker are strings
sp500_aini_to_ret_sub["Period"] = sp500_aini_to_ret_sub["Period"].astype(str)
sp500_aini_to_ret_sub["Ticker"] = sp500_aini_to_ret_sub["Ticker"].astype(str)

# Total number of models tested
total = sp500_aini_to_ret_sub["joint rej. (α=0.1)"].count()

# Number of rejections (both bootstrap + HC3 significant)
n_reject = sp500_aini_to_ret_sub["joint rej. (α=0.1)"].sum()

# Rejection rate
rejection_rate = n_reject / total * 100

print(f"Total models: {total}")
print(f"Both-method rejections: {n_reject}")
print(f"Rejection rate: {rejection_rate:.2f}%")


In [ ]:
# calculate rejection rate 

# Make sure Year and Ticker are strings
sp500_ret_to_aini_sub["Period"] = sp500_ret_to_aini_sub["Period"].astype(str)
sp500_ret_to_aini_sub["Ticker"] = sp500_ret_to_aini_sub["Ticker"].astype(str)

# Total number of models tested
total = sp500_ret_to_aini_sub["joint rej. (α=0.1)"].count()

# Number of rejections (both bootstrap + HC3 significant)
n_reject = sp500_ret_to_aini_sub["joint rej. (α=0.1)"].sum()

# Rejection rate
rejection_rate = n_reject / total * 100

print(f"Total models: {total}")
print(f"Both-method rejections: {n_reject}")
print(f"Rejection rate: {rejection_rate:.2f}%")


In [ ]:
# subset significant results
sp500_aini_to_ret_sig = sp500_aini_to_ret_sub[sp500_aini_to_ret_sub["joint rej. (α=0.1)"] == True]

# drop 0 value cols
sp500_aini_to_ret_sig = sp500_aini_to_ret_sig.dropna(axis=1,how="all")

# Coerce β columns to numeric once
sp500_aini_to_ret_sig[["β1","β2","β3"]] = (
    sp500_aini_to_ret_sig[["β1","β2","β3"]]
    .apply(pd.to_numeric, errors="coerce")
)

# group by ticker
model_group_tickers = (
    sp500_aini_to_ret_sig
    .groupby(["Ticker"])
    .size()
    .reset_index(name="jointly rejected at α=0.1")
    .sort_values(by="jointly rejected at α=0.1",ascending=False)
)


# print
print(model_group_tickers)

In [ ]:
# group by period
model_group_period = (
    sp500_aini_to_ret_sig
    .groupby(["Period"])
    .size()
    .reset_index(name="jointly rejected at α=0.1")
    .sort_values(by="jointly rejected at α=0.1",ascending=False)
)


# print
print(model_group_period)


In [ ]:
# group by period
model_group_tickers = (
    sp500_aini_to_ret_sig
    .groupby(["Period","Ticker"])
    .size()
    .reset_index(name="jointly rejected at α=0.1")
    .sort_values(by="Ticker",ascending=True)
)


# print
print(model_group_tickers)

In [ ]:
model_group_measure = (
    sp500_aini_to_ret_sig
    .groupby(["AINI_variant"])
    .size()
    .reset_index(name="n_variants")
    .sort_values(by="n_variants",ascending=False)
)

print(model_group_measure)

In [ ]:
# find distinctions between models
keys = ["Ticker", "Period"]  
models = ["w0", "w1", "w2", "custom"]               

common_dfs = []
left_only_dfs = []
right_only_dfs = []

for m1, m2 in combinations(models, 2):
    df1 = sp500_aini_to_ret_sig.loc[sp500_aini_to_ret_sig["Model"] == m1, keys].drop_duplicates()
    df2 = sp500_aini_to_ret_sig.loc[sp500_aini_to_ret_sig["Model"] == m2, keys].drop_duplicates()

    # intersection
    common = df1.merge(df2, on=keys, how="inner")
    if not common.empty:
        common = common.assign(Model_pair=f"{m1}&{m2}")
        common_dfs.append(common)

    # only in left / only in right
    cmp = df1.merge(df2, on=keys, how="outer", indicator=True)
    left_only  = cmp.loc[cmp["_merge"] == "left_only",  keys].assign(only=m1)
    right_only = cmp.loc[cmp["_merge"] == "right_only", keys].assign(only=m2)

    if not left_only.empty:
        left_only_dfs.append(left_only)
    if not right_only.empty:
        right_only_dfs.append(right_only)

# Concatenate
common_all = pd.concat(common_dfs, ignore_index=True) if common_dfs else pd.DataFrame(columns=keys+["Model_pair"])
left_only_all = pd.concat(left_only_dfs, ignore_index=True) if left_only_dfs else pd.DataFrame(columns=keys+["only"])
right_only_all = pd.concat(right_only_dfs, ignore_index=True) if right_only_dfs else pd.DataFrame(columns=keys+["only"])

right_only_all

In [ ]:
# find distinctions between models
keys = ["Ticker", "Period"]  
models = ["w0", "w1", "w2", "custom"]               

common_dfs = []
left_only_dfs = []
right_only_dfs = []

for m1, m2 in combinations(models, 2):
    df1 = sp500_aini_to_ret_sig.loc[sp500_aini_to_ret_sig["Model"] == m1, keys].drop_duplicates()
    df2 = sp500_aini_to_ret_sig.loc[sp500_aini_to_ret_sig["Model"] == m2, keys].drop_duplicates()

    # intersection
    common = df1.merge(df2, on=keys, how="inner")
    if not common.empty:
        common = common.assign(Model_pair=f"{m1}&{m2}")
        common_dfs.append(common)

    # only in left / only in right
    cmp = df1.merge(df2, on=keys, how="outer", indicator=True)
    left_only  = cmp.loc[cmp["_merge"] == "left_only",  keys].assign(only=m1)
    right_only = cmp.loc[cmp["_merge"] == "right_only", keys].assign(only=m2)

    if not left_only.empty:
        left_only_dfs.append(left_only)
    if not right_only.empty:
        right_only_dfs.append(right_only)

# Concatenate
common_all = pd.concat(common_dfs, ignore_index=True) if common_dfs else pd.DataFrame(columns=keys+["Model_pair"])
left_only_all = pd.concat(left_only_dfs, ignore_index=True) if left_only_dfs else pd.DataFrame(columns=keys+["only"])
right_only_all = pd.concat(right_only_dfs, ignore_index=True) if right_only_dfs else pd.DataFrame(columns=keys+["only"])

# Ensure uniqueness per model
base = sp500_aini_to_ret_sig[["Model", *keys]].drop_duplicates()
models = sorted(base["Model"].unique())

# Build set of tuples for each model
model_sets = {
    m: set(map(tuple, base.loc[base["Model"] == m, keys].to_numpy()))
    for m in models
}

# Precompute sizes
sizes = {m: len(model_sets[m]) for m in models}

# Row-normalized intersection fractions
frac = pd.DataFrame(index=models, columns=models, dtype=float)

# pure intersection counts
counts = pd.DataFrame(index=models, columns=models, dtype=int)

# Jaccard index: |A∩B| / |A∪B|
jaccard = pd.DataFrame(index=models, columns=models, dtype=float)

for m1 in models:
    S1 = model_sets[m1]
    n1 = sizes[m1]
    for m2 in models:
        S2 = model_sets[m2]
        inter = len(S1 & S2)
        union = len(S1 | S2)
        counts.loc[m1, m2] = inter
        frac.loc[m1, m2] = inter / n1 if n1 > 0 else float("nan")
        jaccard.loc[m1, m2] = inter / union if union > 0 else float("nan")

# percentages
frac_pct = (frac * 100).round(1)

# Example: show the row-normalized fraction matrix first
print("Row-normalized intersection fraction (|A∩B| / |A|):")
print(frac.round(3))
print("\nIntersection counts:")
print(counts)
print("\nJaccard index:")
print(jaccard.round(3))


In [ ]:
# investigate groups by Model
model_group_model = (
    sp500_aini_to_ret_sig
    .groupby(["Model"])
    .size()
    .reset_index(name="n_variants")
    .sort_values(by="n_variants",ascending=False)
)
print(model_group_model)
sp500_aini_to_ret_sig

In [ ]:

# plot distribution of betas
# defines paths
out_path = root_dir / "reports" / "figures" / "distribution_of_betas.png"
out_path.parent.mkdir(parents=True, exist_ok=True)

# Config (
plt.rcParams.update({
    "figure.dpi": 100,
    "savefig.dpi": 300,
    "font.size": 10,
    "axes.titlesize": 11,
    "axes.labelsize": 10,
    "xtick.labelsize": 9,
    "ytick.labelsize": 9,
    "legend.fontsize": 9,
})


# b columns (exclude b0)
beta_cols = [c for c in sp500_aini_to_ret_sig.columns if c.startswith("β") and c != "β₀"]

# Ensure consistent model order
models = ["w0", "w1", "w2", "custom"]

# Symmetric x-axis centered on 0 across ALL models
all_betas = sp500_aini_to_ret_sig[beta_cols].to_numpy().astype(float).ravel()
all_betas = all_betas[~np.isnan(all_betas)]
lim = float(max(abs(all_betas.min()), abs(all_betas.max())))
xlim = (-lim, lim)

# Fixed y-limit
ymax = 23
bins = 40

# Plot
fig, axes = plt.subplots(2, 2, figsize=(7.2, 5.6), sharex=True, sharey=True)  # ~single-column friendly
axes = axes.flatten()

for i, model in enumerate(models):
    ax = axes[i]
    subset = sp500_aini_to_ret_sig.loc[sp500_aini_to_ret_sig["Model"] == model, beta_cols]

    # Plot combined histograms 
    subset.plot.hist(
        bins=bins,
        range=xlim,         
        alpha=0.6,
        ax=ax,
        edgecolor="black",
        legend=True,         
    )

    # Titles / axes
    ax.set_title(f"Model: {model}")
    ax.set_xlim(xlim)
    ax.set_ylim(0, ymax)
    ax.set_ylabel("Counts")         
    ax.grid(alpha=0.25, linestyle=":", linewidth=0.8)

    # Means (β̄) per β, upper-right corner
    means = subset[beta_cols].mean().values

    # Map β-column names 
    beta_labels = []
    for j, col in enumerate(beta_cols, start=1):
        beta_labels.append(fr"$\bar{{\beta}}_{{{j}}}$={means[j-1]:.3f}")
    means_str = ", ".join(beta_labels)

    ax.text(
        0.98, 0.97, means_str,
        transform=ax.transAxes,
        ha="right", va="top",
        fontsize=8.5,
        bbox=dict(facecolor="white", edgecolor="none", alpha=0.7, boxstyle="round,pad=0.2")
    )


# Ensure common x-label 
fig.text(0.5, 0.02, "", ha="center", fontsize=10)

# Grab labels from last axes 
handles, labels = axes[-1].get_legend_handles_labels()
for ax in axes:
    ax.legend_.remove()
fig.legend(handles, labels, title="Coefficient", loc="lower center", ncol=len(beta_cols), frameon=False)

# Tight layout 
plt.tight_layout(rect=[0.04, 0.07, 1, 0.98])

# Save
fig.savefig(out_path, bbox_inches="tight", facecolor="white")
plt.close(fig)

print(f"Saved: {out_path}")
sp500_aini_to_ret_sig

In [ ]:
# sort by betas
sp500_aini_to_ret_sort = sp500_aini_to_ret_sig.assign(abs_β1=lambda x: x["β1"].abs()).sort_values("abs_β1", ascending=False)
sp500_aini_to_ret_sort_cut = sp500_aini_to_ret_sort.iloc[0:20]

# beautify 


In [ ]:
# sorted latex output
export_regression_table(
    df=sp500_aini_to_ret_sort_cut,
    title="Granger-Causality, jointly significant results (AINI $\\to$ Returns, controlled for S\\&P~500). \\textit{Source:} Own.",
    output_filename="gc_sp500_aini_to_ret_sort_beta_cut",  
    output_format="tex",
    latex_env="tabular",          
    include_caption_label=False,   
    wrap_headers=False,           
    coef_digits=3,
    p_digits=3,
    tabcolsep_pt=2.0,
    font_size_cmd="scriptsize",   
)


In [ ]:
export_regression_table(
    df=sp500_aini_to_ret_sort,
    title="Granger-Causality, jointly significant results (AINI → Returns, controlled for S&P 500)",
    output_filename="gc_sp500_aini_to_ret_sort_beta",
    output_format="tex",
    latex_env="longtable",         
    include_caption_label=True,    
    font_size_cmd="scriptsize",
    tabcolsep_pt=2.0,
    coef_digits=3,
    p_digits=3
)


In [ ]:
export_regression_table(
    df=sp500_aini_to_ret_sort_cut,
    title="Granger-Causality, jointly significant results (AINI → Returns, controlled for S&P 500)",
    output_filename="gc_sp500_aini_to_ret_sort_beta_cut",
    output_format="tex",
    latex_env="tabular",           
    include_caption_label=False,   
    font_size_cmd="scriptsize",
    tabcolsep_pt=2.0
)


Controlling for VIX

In [ ]:
# save merged results
gc_all_results_VIX.to_csv(var_path / "granger_causality_VIX.csv", index=False)

# define table path
table_path = root_dir / "reports" / "tables"

# Export as HTML for online appendix
gc_all_results.to_html(table_path / "granger_causality_VIX.html", index=False)

In [ ]:
# subset by direction
vix_aini_to_ret = gc_all_results_VIX[gc_all_results_VIX["Direction"] == "AINI_to_RET"]
vix_ret_to_aini = gc_all_results_VIX[gc_all_results_VIX["Direction"] == "RET_to_AINI"]

# cols to keep
keep_a2r = [
    "Model", "AINI_variant", "Ticker", "Period", "Lags", "β₀",
    "β1", "β2", "β3",
    "γ1", "γ2", "γ3",
    "BH empirical p", "BH analytical p", "joint rej. (α=0.1)"
]

keep_r2a = [
    "Model", "AINI_variant", "Ticker", "Period", "Lags", "β₀",
    "β1", "β2", "β3",
    "γ1", "γ2", "γ3",
    "BH empirical p", "BH analytical p", "joint rej. (α=0.1)"
]

# subset
vix_aini_to_ret_sub = vix_aini_to_ret[keep_a2r]
vix_ret_to_aini_sub = vix_ret_to_aini[keep_r2a]
vix_aini_to_ret

In [ ]:
# calculate rejection rate 

# Make sure Year and Ticker are strings
vix_aini_to_ret_sub["Period"] = vix_aini_to_ret_sub["Period"].astype(str)
vix_aini_to_ret_sub["Ticker"] = vix_aini_to_ret_sub["Ticker"].astype(str)

# Total number of models tested
total = vix_aini_to_ret_sub["joint rej. (α=0.1)"].count()

# Number of rejections (both bootstrap + HC3 significant)
n_reject = vix_aini_to_ret_sub["joint rej. (α=0.1)"].sum()

# Rejection rate
rejection_rate = n_reject / total * 100

print(f"Total models: {total}")
print(f"Both-method rejections: {n_reject}")
print(f"Rejection rate: {rejection_rate:.2f}%")

Investigate assets

In [ ]:
# beautify
vix_aini_to_ret_sub["Ticker"] = vix_aini_to_ret_sub["Ticker"].replace({"TSM": "TSMC"})
vix_aini_to_ret_sub["Period"] = vix_aini_to_ret_sub["Period"].replace({"2023_24": "2023-2024"})
vix_aini_to_ret_sub["Period"] = vix_aini_to_ret_sub["Period"].replace({"2024_25": "2024-2025"})
vix_aini_to_ret_sub["Period"] = vix_aini_to_ret_sub["Period"].replace({"2023_24_25": "2023-2025"})


Controlled for VIX

Investigate results